To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [ ]:
%%capture
!pip install --no-deps git+https://github.com/huggingface/transformers.git unsloth_zoo# Need main branch for Liquid LFM2 models
!pip install --no-deps causal-conv1d==1.5.0.post8 # Install Mamba kernels

### Unsloth

In [19]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/LFM2-1.2B-unsloth-bnb-4bit",
    "unsloth/LFM2-700M-unsloth-bnb-4bit",
    "unsloth/LFM2-350M-unsloth-bnb-4bit",

    # Full 16bit unquantized models
    "unsloth/LFM2-1.2B",
    "unsloth/LFM2-700M",
    "unsloth/LFM2-350M",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/LFM2-1.2B",
    dtype = None, # None for auto detection
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.11.3: Fast Lfm2 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [20]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # LFM for now is just text only
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `LFM` format for conversation style finetunes. We use [miriad/miriad-4.4M](https://huggingface.co/datasets/miriad/miriad-4.4M) dataset and we will add conversations style into the dataset. LFM renders multi turn conversations like below:

```
<|startoftext|><|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>
```

In [21]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Hello!"},
    {"role" : "assistant", "content" : "Hey there!"}
], tokenize = False)

'<|startoftext|><|im_start|>user\nHello!<|im_end|>\n<|im_start|>assistant\nHey there!<|im_end|>\n'

We get the first 10000 rows of the dataset

In [2]:
from datasets import load_dataset, Dataset

# 1. Stream the dataset
dataset = load_dataset("PleIAs/SYNTH", split="train", streaming=True)

# 2. Filter only German
def is_german(example):
    lang = example.get("language", "")
    return lang and "de" in lang.lower()

de_stream = dataset.filter(is_german)

# 3. Transform to conversations format
def transform(example):
    q = example.get("query", "").strip()
    a = example.get("synthetic_answer", "").strip()
    return {
        "conversations": [
            {"role": "user", "content": q},
            {"role": "assistant", "content": a}
        ]
    }

# 4. Define generator *function* (not generator object)
def gen():
    count = 0
    for ex in de_stream:
        yield transform(ex)
        count += 1
        if count >= 10000:   # limit to 10000 if desired
            break

# 5. Build the in-memory dataset
de_synth = Dataset.from_generator(gen)

print(de_synth)
print(de_synth[0])

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['conversations'],
    num_rows: 10000
})
{'conversations': [{'content': 'Wie genau interagieren die Wärmegradienten zwischen dem kalten nordwestlichen Wintermonsun und dem warmen Sommerwind mit der Topographie der japanischen Inseln, um diese speziellen Kármán-Wirbelwolken im Meer zu erzeugen?', 'role': 'user'}, {'content': 'Die Wärmegradienten zwischen dem kalten nordwestlichen Wintermonsun und dem warmen Sommerwind spielen eine entscheidende Rolle bei der Entstehung der charakteristischen Kármán-Wirbelwolken über dem Japanischen Meer.\n\nIm Winter bringt der starke nordwestliche Monsun mit Geschwindigkeiten von 12-15 m/s kalte, trockene kontinentale Luft aus Richtung Sibirien. Diese Luftmasse führt zu markanten Temperaturunterschieden zwischen den kalten nördlichen Gebieten mit durchschnittlichen Lufttemperaturen von -20°C im Januar/Februar und den wärmeren südlichen Regionen mit etwa 5°C. Der Wintermonsun verstärkt zudem die Verdunstung und Oberflächenwasser

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [3]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(de_synth)

ModuleNotFoundError: No module named 'unsloth'

Let's see how row 100 looks like!

In [26]:
dataset[100]

NotImplementedError: Subclasses of Dataset should implement __getitem__.

We now have to apply the chat template for `LFM` onto the conversations, and save it to `text`. We also remove the BOS token otherwise we'll get double BOS tokens!

In [9]:
def formatting_prompts_func(examples):
    texts = tokenizer.apply_chat_template(
        examples["conversations"],
        tokenize = False,
        add_generation_prompt = False,
    )
    return { "text" : [x.removeprefix(tokenizer.bos_token) for x in texts] }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
dataset[0]["text"]

'<|im_start|>user\nWhat is the overall response rate of trastuzumab in adult relapsed/refractory HER2-positive B-ALL patients?<|im_end|>\n<|im_start|>assistant\nThe overall response rate of trastuzumab in adult relapsed/refractory HER2-positive B-ALL patients is 13%.<|im_end|>\n'

**Test the model before Training**

Now let's test the model before training.

In [11]:
messages = [{
    "role": "user",
    "content": """What are the potential future directions for treating HER2-positive B-ALL?"""
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 2048, # Increase for longer outputs!
    # Recommended Liquid settings!
    temperature = 0.3, min_p = 0.15, repetition_penalty = 1.05,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

HER2-positive diffuse large B-cell lymphoma (DLBCL) is a challenging subtype of non-Hodgkin lymphoma, and its treatment landscape is evolving rapidly. Given the aggressive nature of this subtype and the limitations of current therapies, several promising future directions are being explored to improve outcomes for patients with HER2-positive B-ALL:

1. **Targeted Therapies**:
   - **Trastuzumab Deruxtecan (Enhertu)**: This antibody-drug conjugate has shown remarkable efficacy in HER2-positive cancers, including some subtypes of lymphoma. Its high potency and favorable safety profile make it a promising candidate for HER2-positive B-ALL.
   - **T-DM1 (T-DM1/Enhertu)**: Another trastuzumab derivative, T-DM1, is also being studied for its effectiveness in HER2-positive lymphomas, including B-cell lymphomas.

2. **Combination Therapies**:
   - Combining HER2-targeted agents with other therapies, such as BTK inhibitors (e.g., ibrutinib), PI3K inhibitors (e.g., idelalisib), or CD20 monoclona

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [12]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/10000 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [13]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=6):   0%|          | 0/10000 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again.

In [14]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|startoftext|><|im_start|>user\nHow do radiation-resistant leukemia cell lines differ from the parental cell line in terms of DNA damage response and apoptosis?<|im_end|>\n<|im_start|>assistant\nRadiation-resistant leukemia cell lines, such as HL60/RX and HL60/ADR, show increased expression of SMO and Gli-1, which are molecules involved in the Hedgehog (Hh) signaling pathway. These cell lines exhibit high survival rates after irradiation and have reduced levels of DNA double-strand breaks (DSBs) and apoptosis compared to the parental cell line. This suggests that the activation of Hh signaling in these cell lines contributes to their radiation resistance.<|im_end|>\n'

Now let's print the masked out example - you should see only the answer is present:

In [15]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                               Radiation-resistant leukemia cell lines, such as HL60/RX and HL60/ADR, show increased expression of SMO and Gli-1, which are molecules involved in the Hedgehog (Hh) signaling pathway. These cell lines exhibit high survival rates after irradiation and have reduced levels of DNA double-strand breaks (DSBs) and apoptosis compared to the parental cell line. This suggests that the activation of Hh signaling in these cell lines contributes to their radiation resistance.<|im_end|>\n'

In [16]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.273 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 1,250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 9,142,272 of 1,179,482,880 (0.78% trained)


Step,Training Loss
1,2.968200
2,2.755500
3,3.009100
4,2.893600
5,2.834600
6,3.052300
7,3.132400
8,2.810200
9,3.071200
10,2.567200


Unsloth: Will smartly offload gradients to save VRAM!


KeyboardInterrupt: 

In [18]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'trainer_stats' is not defined

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [ ]:
messages = [{
    "role": "user",
    "content": """What are the potential future directions for treating HER2-positive B-ALL?""",
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 2048, # Increase for longer outputs!
    # Recommended Liquid settings!
    temperature = 0.3, min_p = 0.15, repetition_penalty = 1.05,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The potential future directions for treating HER2-positive B-ALL include the development of targeted therapies that specifically inhibit HER2 signaling. This could involve the use of monoclonal antibodies, small molecule inhibitors, or other agents that selectively target HER2. Additionally, there is a need for better understanding of the mechanisms of resistance to HER2-targeted therapies and strategies to overcome this resistance.<|im_end|>


In [ ]:
messages = [{
    "role": "user",
    "content": """What are some factors that determine the prognosis of patients undergoing bowel resection for colorectal cancer?""",
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Liquid settings!
    temperature = 0.3, min_p = 0.15, repetition_penalty = 1.05,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Several factors have been identified as predictors of the prognosis of patients undergoing bowel resection for colorectal cancer. These include the presence of lymph node metastasis, the stage of the disease at the time of surgery, and the presence of extraintestinal metastases. Additionally, the presence of a positive lymph node margin or the presence of a tumor in the rectum are also associated with a worse prognosis.<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    from transformers import Lfm2ForCausalLM
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        auto_model = Lfm2ForCausalLM,
        load_in_4bit = load_in_4bit,
    )
    FastModel.for_inference(model) # Enable native 2x faster inference

messages = [{
    "role": "user",
    "content": "How do I code up a transformer?",
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Liquid settings!
    temperature = 0.3, min_p = 0.15, repetition_penalty = 1.05,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

To code up a transformer, you can follow these steps:

1. Import the necessary libraries:
```python
import torch
from transformers import TFAutoModelForSequenceClassification, TFAutoModelForTokenClassification
```

2. Load the pre-trained model and tokenizer:
```python
model_name = "bert-base-uncased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = TFAutoModelForTokenClassification.from_pretrained(model_name)
```

3. Define the input and output classes:
```python
input_class


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
